<a href="https://colab.research.google.com/github/salonigupta1/Fake-News-Classifier-using-LSTM/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [137]:
import pandas as pd

In [138]:
df=pd.read_csv('spam.csv', encoding = 'latin-1')

In [139]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [140]:
df['v1']

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [141]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
 
df['v1']= label_encoder.fit_transform(df['v1'])
 
df['v1'].unique()

array([0, 1])

In [142]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,0,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,0,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,0,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [143]:
###Drop Nan Values
df=df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)


In [144]:
df

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [145]:
## Get the Independent Features

X=df['v2']

In [146]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [147]:
## Get the Dependent features
y=df['v1']

In [148]:
print(y)

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: v1, Length: 5572, dtype: int64


In [149]:
X.shape

(5572,)

In [150]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [151]:
y.shape

(5572,)

In [152]:
import tensorflow as tf

In [153]:
tf.__version__

'2.6.0'

In [154]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [155]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [156]:
messages=X.copy()

In [157]:
messages[1]

'Ok lar... Joking wif u oni...'

In [158]:
import nltk
import re
from nltk.corpus import stopwords

In [159]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [160]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
8

In [161]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

In [163]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1455,
  996,
  537,
  426,
  939,
  103,
  682,
  2996,
  4327,
  300,
  1073,
  4370,
  1699,
  2530,
  4998,
  2684],
 [2929, 3497, 1996, 1506, 3716, 2325],
 [3451,
  1508,
  2857,
  3409,
  2826,
  347,
  814,
  1410,
  386,
  384,
  1195,
  2664,
  347,
  2681,
  1508,
  2040,
  3329,
  3117,
  2239,
  4180,
  3437],
 [3716, 2702, 2051, 247, 1565, 3716, 4180, 2446, 2051],
 [4173, 1724, 496, 2589, 2647, 2875, 3238],
 [98,
  2869,
  952,
  2740,
  140,
  3185,
  892,
  3335,
  2601,
  2332,
  2929,
  2368,
  3329,
  2110,
  4101,
  4310],
 [2821, 1751, 892, 46, 1516, 892, 1812, 469],
 [1972,
  690,
  4564,
  4564,
  118,
  2708,
  2149,
  2601,
  4031,
  2292,
  1308,
  2137,
  4339,
  4825,
  2292],
 [4336,
  2675,
  201,
  1923,
  2069,
  497,
  2053,
  3727,
  157,
  2505,
  157,
  1252,
  1572,
  4103,
  1295],
 [1740,
  3178,
  3716,
  4952,
  1934,
  1145,
  169,
  728,
  1740,
  1061,
  3451,
  2505,
  1740,
  1145,
  4183,
  3451],
 [1214, 49, 946, 1698, 3709, 3365, 4057, 4

### Embedding Representation

In [164]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2530 4998 2684]
 [   0    0    0 ... 1506 3716 2325]
 [1508 2857 3409 ... 2239 4180 3437]
 ...
 [   0    0    0 ... 4918 1526 4613]
 [   0    0    0 ... 4629 1540 3451]
 [   0    0    0 ... 1234  323 4190]]


In [165]:
embedded_docs[0]

array([   0,    0,    0,    0, 1455,  996,  537,  426,  939,  103,  682,
       2996, 4327,  300, 1073, 4370, 1699, 2530, 4998, 2684], dtype=int32)

In [166]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [167]:
len(embedded_docs),y.shape

(5572, (5572,))

In [168]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [169]:
X_final.shape,y_final.shape

((5572, 20), (5572,))

In [170]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [171]:
y_test

array([0, 0, 1, ..., 0, 0, 1])

In [172]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
59/59 [==============================] - 3s 21ms/step - loss: 0.3208 - accuracy: 0.8878 - val_loss: 0.1553 - val_accuracy: 0.9772
Epoch 2/10
59/59 [==============================] - 1s 13ms/step - loss: 0.0770 - accuracy: 0.9810 - val_loss: 0.0624 - val_accuracy: 0.9848
Epoch 3/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0356 - accuracy: 0.9904 - val_loss: 0.0527 - val_accuracy: 0.9864
Epoch 4/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0141 - accuracy: 0.9960 - val_loss: 0.0550 - val_accuracy: 0.9842
Epoch 5/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0096 - accuracy: 0.9976 - val_loss: 0.0608 - val_accuracy: 0.9864
Epoch 6/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0047 - accuracy: 0.9992 - val_loss: 0.0650 - val_accuracy: 0.9826
Epoch 7/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0044 - accuracy: 0.9992 - val_loss: 0.0683 - val_accuracy: 0.9864
Epoch 8/10
59

### Adding Dropout 

In [173]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [174]:
y_pred=model.predict(X_test)

In [180]:
y_pred

array([[0.49753645],
       [0.5016023 ],
       [0.4993916 ],
       ...,
       [0.49566168],
       [0.4978062 ],
       [0.50338614]], dtype=float32)

In [184]:
y_pred = np.argmax(y_pred, axis=1) 

In [185]:
from sklearn.metrics import confusion_matrix

In [186]:
y_test

array([0, 0, 1, ..., 0, 0, 1])

In [187]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [188]:
confusion_matrix(y_test,y_pred)

array([[1587,    0],
       [ 252,    0]])

In [189]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8629690048939641